<a href="https://colab.research.google.com/github/cosmina98/PhD/blob/main/MLP_Wrapped_update/Wrapped_up_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install torchmetrics --quiet

     |████████████████████████████████| 419 kB 8.9 MB/s 


In [ ]:

!pip install -U scikit-learn --user



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.anaconda.org/scipy-wheels-nightly/simple


In [ ]:
# libary imports 
import os
import torch
from torch import nn
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm

import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import time

import itertools
from functools import reduce
import operator
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix
from torch.utils.data import Subset

from torch.optim.adadelta import Adadelta
from torch.optim.adagrad import Adagrad
from torch.optim.adam import Adam
from torch.optim.adamax import Adamax
from torch.optim.adamw import AdamW
from torch.optim.optimizer import Optimizer
from torch.optim.sgd import SGD



plt.rcParams['figure.figsize'] = [10, 20]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Main program

In [ ]:
def get_classification_report(y, y_pred):
        y_prime = np.stack([np.stack([d for d in d_]) for d_ in y]).flatten()
        y_pred_prime=np.stack([np.stack([d for d in d_]) for d_ in y_pred]).flatten()   
        print(classification_report(y_prime, y_pred_prime))


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


def get_confusion_matrix(y, y_pred, plot=True):
      y_prime = np.stack([np.stack([d for d in d_]) for d_ in y]).flatten()
      y_pred_prime=np.stack([np.stack([d for d in d_]) for d_ in y_pred]).flatten()
      cnf_matrix = confusion_matrix(y_prime, y_pred_prime,labels=np.arange(10))
      np.set_printoptions(precision=2)
      # Plot non-normalized confusion matrix
      if plot==True:
        plt.figure()
        plot_confusion_matrix(cnf_matrix, classes=['plane', 'car', 'bird', 'cat',
                            'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
                              title='Confusion matrix, without normalization')
          

In [ ]:


# Prepare CIFAR-10 dataset
trainset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

testset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))


#print(images, labels)

def my_fun(dataset):
    X=[]
    y=[]
    for feature, label in iter(dataset):
        X.append(feature)
        y.append(label)
    return torch.stack(X).numpy(), y


#get  data as numpy arrays
X_train, y_train = my_fun(trainset)
X_test, y_test =my_fun(testset)


Files already downloaded and verified
Files already downloaded and verified


In [58]:
class MLPClassifer(nn.Module):
      '''
        Multilayer Perceptron.
      '''
      def __init__(
        self,
        input_dim=trainset.data[1].shape,
        output_dim=len(torch.unique(torch.tensor(trainset.targets))),
        first_hidden_dimension=64,
        #if you only want to add  a single hidden layer you do the following:
       # hidden_layers=((32),)
        hidden_layers=((32),),
        ):
        super().__init__()
        #self.modules1 = torch.nn.ModuleList([])
        self.residual=False
        self.hidden_layer_num=len(hidden_layers)
        self.d_in=reduce(operator.mul, input_dim)
        if self.hidden_layer_num>0:
            self.h_0=first_hidden_dimension
            self.h_n=hidden_layers[-1]
 
        else: 
            self.h_0=self.h_n=first_hidden_dimension
      

        self.linear1=nn.Linear(self.d_in,  self.h_0)
        self.relu=torch.nn.ReLU()
        self.hidden_linear=[]
        if self.hidden_layer_num>0:
          in_channels=self.h_0
          for h_dim in list(hidden_layers):
              #self.modules1.append(nn.Linear(in_channels, in_channels))
              self.hidden_linear.append(torch.nn.Linear(in_channels, h_dim))
              in_channels = h_dim
        self.hidden_linear=nn.ModuleList(self.hidden_linear)
        self.linear2=nn.Linear( self.h_n, output_dim)
        #self.modules1.append(nn.Linear( self.h_n, output_dim))
     

      def forward(self, x):
        x=x.view(-1,self.d_in)
        x=self.linear1(x)
        x=self.relu(x)
        if self.hidden_layer_num>0:
          for linear in self.hidden_linear:
            if self.residual:
              x=self.relu(linear(x)) + x
          else:
              x = self.relu(linear(x))
        return  F.log_softmax(self.linear2(x), dim=1)

class Optimiser(object):
        def __init__(self,name='Adam',classifier=MLPClassifer(),learning_rate=1e-3,  betas=(0.9,0.99) , eps=1e-8, weight_decay=0, amsgrad=False,
      momentum=0.9, nesterov=True, dampening=0, maximize=False,
      rho=0.9,lr_decay=0,initial_accumulator_value=0,capturable=False,
       max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=None,
       lambd=0.0001, alpha=0.75, t0=1000000.0, momentum_decay=0.004,centered=False,
       etas=(0.5, 1.2), step_sizes=(1e-06, 50)):
          self.learning_rate=learning_rate 
          self.betas= betas
          self.eps= eps
          self.weight_decay=weight_decay
          self.amsgrad=amsgrad 
          self.momentum=momentum
          self.nesterov=nesterov
          self.dampening=dampening
          self.maximize=maximize
          self.classifier=classifier
          self.name=name
          self.optimiser=self.set_optimizer(self.name)
          self.rho=rho
          self.lr_decay=lr_decay
          self.capturable=capturable

          #lbfgs
          self.max_iter=max_iter
          self.max_eval=max_eval
          self.tolerance_grad=tolerance_grad
          self.tolerance_change=tolerance_change
          self.history_size=history_size
          self.line_search_fn=line_search_fn

          #asgd
          self.lambd=lambd
          self.alpha=alpha
          self.t0=t0
          
          #Nadam
          self.momentum_decay=momentum_decay

          self.centered=centered
          self.etas=etas
          self.step_sizes=step_sizes

        def set_optimizer(self, name):
            if name.lower().strip()=='adam':
                        self.optimiser=torch.optim.Adam(self.classifier.parameters(),lr=self.learning_rate, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay, amsgrad=self.amsgrad,maximize=self.maximize)
            elif name.lower().strip()=='sgd':
                        self.optimiser=torch.optim.SGD(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, momentum=0.9,nesterov=self.nesterov , dampening=self.dampening,maximize=self.maximize)
            elif name.lower().strip()=='adadelta':
                        self.optimiser=torch.optim.Adadelta(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, eps=self.eps, rho=self.rho,maximize=self.maximize)
            elif name.lower().strip()=='adagrad':
                        self.optimiser=torch.optim.Adagrad(self.classifier.parameters(),lr=self.learning_rate, eps=self.eps, weight_decay=self.weight_decay, maximize=self.maximize,lr_decay=self.lr_decay, initial_accumulator_value=self.initial_accumulator_value)
            elif name.lower().strip()=='adamw':
                        self.optimiser=torch.optim.AdamW(self.classifier.parameters(),lr=self.learning_rate, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay, amsgrad=self.amsgrad,maximize=self.maximize, capturable=self.capturable)
            elif name.lower().strip()=='adamax':
                        self.optimiser=torch.optim.Adamax(self.classifier.parameters(),lr=self.learning_rate, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay,maximize=self.maximize)
            elif name.lower().strip()=='lbfgs':
                        self.optimiser=torch.optim.LBFGS(self.classifier.parameters(),lr=self.learning_rate, max_iter=self.max_iter,max_eval=self.max_eval, tolerance_grad=self.tolerance_grad,tolerance_change=self.tolerance_change, history_size=self.history_size,line_search_fn=self.line_search_fn)
            elif name.lower().strip()=='asgd':
                        self.optimiser=torch.optim.ASGD(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, lambd=self.lambd, alpha=self.alpha, t0=self.t0 )
            elif name.lower().strip()=='nadam':
                        self.optimiser=torch.optim.NAdam(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, eps=self.eps, betas=self.betas, momentum_decay=self.momentum_decay)
            elif name.lower().strip()=='radam':
                        self.optimiser=torch.optim.RAdam(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, eps=self.eps, betas=self.betas )
            elif name.lower().strip()=='rmsprop':
                        self.optimiser=torch.optim.RMSprop(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, momentum=self.momentum,alpha=self.alpha, eps=self.eps, centered=self.centered)
            elif name.lower().strip()=='rprop':
                        self.optimiser=torch.optim.Rprop(self.classifier.parameters(),lr=self.learning_rate, etas=self.etas, step_sizes=self.step_sizes)
            
            return self.optimiser


class Neural_Network(object):
    
    
    def __init__ (self,batch_size=2000,epochs=2,verbose=True,loss_function = nn.CrossEntropyLoss(),
         optimiser_name='adam',classifier= MLPClassifer()):
      
      self.batch_size=batch_size
      self.epochs=epochs
      
      self.verbose=True
      #used inside the training/ testing loops
      self.running_loss = 0
      self.loss_function = loss_function
      self.classifier= classifier
      
      self.sm = torch.nn.Softmax()

      #default optimiser: adam 
      self.optimiser=Optimiser(optimiser_name,classifier=self.classifier).optimiser
    
    
    def get_params(self): #get parameters
       if (self.optimiser != None) :
         return  (self.props(),self.optimiser.state_dict(), self.classifier.state_dict)
       else: return self.classifier.state_dict() 
  
    def fit(self,X,y):
        train = [*zip(X,y)]
        trainloader =  torch.utils.data.DataLoader(train,batch_size=8, shuffle=False, num_workers=2)
        for epoch in range(self.epochs):  # loop over the dataset 2 times
        
          running_loss =self.running_loss
          for i, data in enumerate(trainloader, 0):
          # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
             
            # zero the parameter gradients
            self.optimiser.zero_grad()

            # forward + compute loss+ backward + optimize
            outputs = self.classifier(inputs)
            loss = self.loss_function(outputs, labels)
            loss.backward()
            self.optimiser.step()
            # print statistics
            running_loss += loss.item()
            if i % self.batch_size == (self.batch_size-1):    # print every "batch_size" mini-batches
                if self.verbose==True:
                  print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / self.batch_size:.3f}')
                running_loss = 0.0

    def predict(self, X_test): #Predict using the multi-layer perceptron classifier.
      test = [*zip(X_test,y_test)]
      testloader = torch.utils.data.DataLoader(test, shuffle=False)
      prediction_list=[]
      # again no gradients needed
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = self.classifier(images)
              estimation, predictions = torch.max(outputs, 1)
              prediction_list.append(predictions)
      return prediction_list #torch.stack(prediction_list)


    def predict_log_proba(self,X):  #	Return the log of probability estimates.
        data = [*zip(X,y_test)]
        loader =  torch.utils.data.DataLoader(data)
        y_prob=self.predict_proba(loader)
        log_proba=np.log(y_prob)
        return log_proba
 

    def predict_proba(self,X_test):	#Probability estimates.
      probabilities_list=[]
      data = [*zip(X_test,y_test)]
      loader =  torch.utils.data.DataLoader(data, shuffle=False)
      # again no gradients needed
      with torch.no_grad():
          for data in loader:
              images, labels = data
              outputs = self.classifier(images)
              probabilities_list.append(self.sm(outputs) )
              estimation, predictions = torch.max(outputs, 1)
      return probabilities_list
      

    def score(self,X,y): #Return the mean accuracy on the given test data and labels.
        prediction_list=[]
        targets_list=[]
        data = [*zip(X,y)]
        loader =  torch.utils.data.DataLoader(data,batch_size=8, shuffle=False, num_workers=2)
        num_classes=len(torch.unique(torch.tensor(y)))
        #set it to true if you have a list of available labels in text format
        labels_available=False
        if labels_available:
            classes = ('plane', 'car', 'bird', 'cat',
                    'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
            # prepare to count predictions for each class
            correct_pred = {classname: 0 for classname in classes}
            total_pred = {classname: 0 for classname in classes}
        #use the class number as key
        else: 
            classes=tuple(np.arange(num_classes))  
            correct_pred = {classname: 0 for classname in np.arange(num_classes)}
            total_pred= {classname: 0 for classname in np.arange(num_classes)}

        # again no gradients needed
        with torch.no_grad():
            for data in loader:
                images, labels = data
                targets_list.append(labels)
                outputs = self.classifier(images)
                estimation, predictions = torch.max(outputs, 1)
                prediction_list.append(predictions)
                # collect the correct predictions for each class
                for label, prediction in zip(labels, predictions):
                    if label == prediction:
                        correct_pred[classes[label]] += 1
                    total_pred[classes[label]] += 1
        accuracies={}
        # print accuracy for each class
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            accuracies[classname]=accuracy
            if self.verbose:
              print(f'Accuracy for class: {classname} is {accuracy:.1f} %')
        return targets_list,prediction_list
        

    def set_params(self, attr, value): #sets parameters
        setattr(self, attr, value)
        
    def props(cls):   
        return [i for i in cls.__dict__.items() if i[:1] != '_']

    def get_classifier(self):
        return self.classifier()


    


#Plot classifier comparaison



In [ ]:
cm = confusion_matrix(y_test_prime, y_pred)

cm_display = ConfusionMatrixDisplay(cm).plot()

NameError: ignored

AttributeError: ignored